In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# Citing papers on GS
df_ref = pd.read_excel("data/JFE_GS_DATA.xlsx")

In [4]:
df_ref = df_ref.iloc[:,1:]

In [5]:
df_ref

,cited_paper,Authors,Journal of Publication,Year of Publication,Cleaned Titles,Author 1 First Name,Author 1 Last Name,Author 2 First Name,Author 2 Last Name,Author 3 First Name,Author 3 Last Name,Author 4 First Name,Author 4 Last Name,Author 5 First Name,Author 5 Last Name,Author 6 First Name,Author 6 Last Name,Author 7 First Name,Author 7 Last Name
0,Delayed crises and slow recoveries,"A Krishnamurthy, W Li",NaN,2020.0,dissecting mechanisms of financial crises: int...,A,Krishnamurthy,W Li,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Delayed crises and slow recoveries,W Li,USC Marshall School of Business Research Paper,2019.0,public liquidity and financial crises,W,Li,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Delayed crises and slow recoveries,"Z Li, S Xu",NaN,NaN,inefficient credit cycles,Z,Li,S,Xu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Learning about the consumption risk exposure o...,"K Li, CY Tsou, C Xu",Journal of Monetary Economics,2023.0,learning and the capital age premium,K,Li,CY,Tsou,C,Xu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Learning about the consumption risk exposure o...,K Schneider,Available at SSRN 4583435,4583.0,"investment, uncertainty, and u-shaped return v...",K,Schneider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410767,Financing patterns around the world: Are small...,F Şahut,NaN,2021.0,kobi̇'lerin finansman sorunları ve çözüm öneri...,F Şahut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410768,Financing patterns around the world: Are small...,C Viorica,Autoreferatul tezei de doctor în științe econo...,2016.0,finanțarea întreprinderilor mici și mijlocii d...,C,Viorica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410769,Financing patterns around the world: Are small...,MA dos Santos,Revista Inteligência Competitiva,2019.0,relação entre estrutura de financiamento e açõ...,MA dos,Santos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410770,Financing patterns around the world: Are small...,粟芳， 初立苹,金融经济学研究,2014.0,中国商业银行综合融资能力测度及影响因素分析,粟芳，,初立苹,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Papers published on JFE 
jfe = pd.read_excel("data/Published_Papers_JFE.xlsx",index_col=0)

In [7]:
jfe

,Title,Authors,Date
0,Delayed crises and slow recoveries,"Xuewen Liu, Pengfei Wang, Zhongchao Yang",February 2024
1,Learning about the consumption risk exposure o...,"Yongjin Kim, Lars-Alexander Kuehn, Kai Li",February 2024
2,Why did shareholder liability disappear?,"David A. Bogle, Gareth Campbell, Christopher C...",February 2024
3,Disagreement about public information quality ...,"Chong Huang, Radhika Lunawat, Qiguang Wang",February 2024
4,Stress tests and model monoculture,"Keeyoung Rhee, Keshav Dogra",February 2024
...,...,...,...
3,The cross-section of expected corporate bond r...,"William R. Gebhardt, Soeren Hvidkjaer, Bhaskar...",January 2005
4,The timing of initial public offerings,"Simon Benninga, Mark Helmantel, Oded Sarig",January 2005
5,The effect of external finance on the equilibr...,"Heitor Almeida, Daniel Wolfenzon",January 2005
6,The “make or take” decision in an electronic m...,"Robert Bloomfield, Maureen O’Hara, Gideon Saar",January 2005


We can make several types of networks: 

1. Author -> paper network 
    * nodes - authors, papers 
    * edges - authors worked on paper x
    * Implication: collaboration network 

2. Citations between papers 
    * nodes - papers (JFE, GS)
    * edges - citations across papers 
    * Implication: citation network 

* Note: We have a lot of features: 
    * Papers with identified authors 
    * University of authors
    * Country of authors 
    * Date of publication 

Citation Network: 

In [10]:
cit_ntwk = df_ref[['Cleaned Titles', 'cited_paper']].rename(columns={'Cleaned Titles': 'citing_paper'})

In [12]:
cit_ntwk

,citing_paper,cited_paper
0,dissecting mechanisms of financial crises: int...,Delayed crises and slow recoveries
1,public liquidity and financial crises,Delayed crises and slow recoveries
2,inefficient credit cycles,Delayed crises and slow recoveries
3,learning and the capital age premium,Learning about the consumption risk exposure o...
4,"investment, uncertainty, and u-shaped return v...",Learning about the consumption risk exposure o...
...,...,...
410767,kobi̇'lerin finansman sorunları ve çözüm öneri...,Financing patterns around the world: Are small...
410768,finanțarea întreprinderilor mici și mijlocii d...,Financing patterns around the world: Are small...
410769,relação entre estrutura de financiamento e açõ...,Financing patterns around the world: Are small...
410770,中国商业银行综合融资能力测度及影响因素分析,Financing patterns around the world: Are small...


In [23]:
merged_df = pd.merge(cit_ntwk, jfe.rename(columns={'Title': 'cited_paper'}), 
                     on='cited_paper',how='right').dropna()

In [35]:
merged_df

,citing_paper,cited_paper,Authors,Date
0,dissecting mechanisms of financial crises: int...,Delayed crises and slow recoveries,"Xuewen Liu, Pengfei Wang, Zhongchao Yang",February 2024
1,public liquidity and financial crises,Delayed crises and slow recoveries,"Xuewen Liu, Pengfei Wang, Zhongchao Yang",February 2024
2,inefficient credit cycles,Delayed crises and slow recoveries,"Xuewen Liu, Pengfei Wang, Zhongchao Yang",February 2024
3,learning and the capital age premium,Learning about the consumption risk exposure o...,"Yongjin Kim, Lars-Alexander Kuehn, Kai Li",February 2024
4,"investment, uncertainty, and u-shaped return v...",Learning about the consumption risk exposure o...,"Yongjin Kim, Lars-Alexander Kuehn, Kai Li",February 2024
...,...,...,...,...
410809,kobi̇'lerin finansman sorunları ve çözüm öneri...,Financing patterns around the world: Are small...,"Thorsten Beck, Asli Demirgüç-Kunt, Vojislav Ma...",September 2008
410810,finanțarea întreprinderilor mici și mijlocii d...,Financing patterns around the world: Are small...,"Thorsten Beck, Asli Demirgüç-Kunt, Vojislav Ma...",September 2008
410811,relação entre estrutura de financiamento e açõ...,Financing patterns around the world: Are small...,"Thorsten Beck, Asli Demirgüç-Kunt, Vojislav Ma...",September 2008
410812,中国商业银行综合融资能力测度及影响因素分析,Financing patterns around the world: Are small...,"Thorsten Beck, Asli Demirgüç-Kunt, Vojislav Ma...",September 2008


In [36]:
merged_df = merged_df[['citing_paper','cited_paper','Date']]

In [37]:
merged_df

,citing_paper,cited_paper,Date
0,dissecting mechanisms of financial crises: int...,Delayed crises and slow recoveries,February 2024
1,public liquidity and financial crises,Delayed crises and slow recoveries,February 2024
2,inefficient credit cycles,Delayed crises and slow recoveries,February 2024
3,learning and the capital age premium,Learning about the consumption risk exposure o...,February 2024
4,"investment, uncertainty, and u-shaped return v...",Learning about the consumption risk exposure o...,February 2024
...,...,...,...
410809,kobi̇'lerin finansman sorunları ve çözüm öneri...,Financing patterns around the world: Are small...,September 2008
410810,finanțarea întreprinderilor mici și mijlocii d...,Financing patterns around the world: Are small...,September 2008
410811,relação entre estrutura de financiamento e açõ...,Financing patterns around the world: Are small...,September 2008
410812,中国商业银行综合融资能力测度及影响因素分析,Financing patterns around the world: Are small...,September 2008


In [45]:
merged_df.to_csv("cit_ntwk.csv")

In [43]:
# Import the NetworkX package
import networkx as nx
from tqdm import tqdm

In [41]:
# Create a directed graph DG
G = nx.DiGraph()

In [44]:
# Set to collect unique nodes
unique_nodes = set(merged_df['citing_paper']) | set(merged_df['cited_paper'])

# Add nodes with attributes and add edges
with tqdm(total=len(unique_nodes)) as pbar:
    for node in unique_nodes:
        G.add_node(node, label='GS' if node in merged_df['citing_paper'].values else 'JFE')
        pbar.update(1)


100%|██████████| 186544/186544 [45:06<00:00, 68.92it/s] 


In [ ]:
# Add edges from DataFrame
with tqdm(total=len(merged_df)) as pbar:
    for _, row in merged_df.iterrows():
        G.add_edge(row['citing_paper'], row['cited_paper'])
        pbar.update(1)

---

Citation network: 

* Nodes - papers (JFE or GS) 
* Edges - citation (directed)

Use GS papers with only the identified authors. 

Should we do the same with identified JFE authors too? 

In [8]:
gs_matches = pd.read_excel("data/JFE_citing_paper_author_matches.xlsx",index_col=0)

In [9]:
gs_matches

,Authors_x,First Name,Middle Name_x,Last Name,Authors_y,University,Source,Middle Name_y,Country
0,A krishnamurthy,Arvind,NaN,Krishnamurthy,Arvind krishnamurthy,stanford university,Abfer,NaN,USA
4,W li,Wendy,C.y.,Li,Wendy c.y. li,Executive director moon economics institute,Cepr,C.y.,Unknown
5,W li,Wendy,C.y.,Li,Wendy c.y. li,Executive director moon economics institute,Cepr,C.y.,Unknown
6,Z li,Zhan,NaN,Li,Zhan li,"Postdoctoral researcher in economics, national...",Cepr,NaN,China
8,K li,Kai,NaN,Li,Kai li,university of british columbia,Abfer,NaN,Canada
...,...,...,...,...,...,...,...,...,...
152487,C eckel,Carsten,NaN,Eckel,Carsten eckel,Professor of economics bibliothek wirtscharfts...,Cepr,NaN,Germany
152525,S winston smith,Stanley,D,Smith,Stanley d smith,university of central florida,Afa,D,USA
152538,R stehrer,Robert,NaN,Stehrer,Robert stehrer,Scientific director the vienna institute for i...,Cepr,NaN,Austria
152566,J kren,Janez,NaN,Kren,Janez kren,Doctoral researcher ku leuven,Cepr,NaN,Unknown
